In [60]:
import os
import pickle
from tqdm import tqdm, trange
from dataclasses import dataclass
import pandas as pd
import json
from dataclasses import dataclass
from typing import Optional
from drl_patches.sparse_autoencoders.get_vectorizer import load_tfidf_vectorizer
from drl_patches.sparse_autoencoders.getting_experiment_config import (
    load_training_indexes,
)
import torch
from drl_patches.logger import logger
from drl_patches.sparse_autoencoders.classical_data_mining import get_metrics


In [ ]:
@dataclass
class Dataset:
    dataset_path: str
    training_idx_path: str

GBUG_DATASET = Dataset(
    dataset_path="../artifacts/gbug-java.csv",
    training_idx_path="../artifacts/gbug-java_train_indexes.json",
)

DEFECT_DATASET = Dataset(
    dataset_path="../artifacts/defects4j.csv",
    training_idx_path="../artifacts/defects4j_train_indexes.json",
)

HUMAN_DATASET = Dataset(
    dataset_path="../artifacts/humaneval.csv",
    training_idx_path="../artifacts/humaneval_train_indexes.json",
)

In [ ]:

def get_testing_dataset(dataset: Dataset):
    """
    Get the testing dataset path and the training indexes path.
    """
    df = pd.read_csv(dataset.dataset_path)
    print(df.shape)
    with open( dataset.training_idx_path, "r") as f:
        training_indices = json.load(f)
    train_df = df.iloc[training_indices]
    test_df = df.drop(train_df.index)

    return train_df, test_df



In [ ]:
@dataclass
class SAE_ACTIVATIONS:
    feature_diff_path: str
    model_path: str
    layer: int
    top_k: int
    dataset: Dataset
    gbug_feature_safe_path: str
    gbug_feature_vuln_path: str
    defects_feature_safe_path: str
    defects_feature_vuln_path: str
    humaneval_feature_safe_path: str
    humaneval_feature_vuln_path: str

SAE_ACTIVATIONS_GBUG_LAYER_1 = SAE_ACTIVATIONS(
    feature_diff_path="../gpt2_gbug-java/layer1/feature_importance_diff.jsonl",
    gbug_feature_safe_path="../gpt2_gbug-java/layer1/feature_importance_safe.jsonl",
    gbug_feature_vuln_path="../gpt2_gbug-java/layer1/feature_importance_vuln.jsonl",
    defects_feature_safe_path="../gpt2_defects4j/layer1/feature_importance_safe.jsonl",
    defects_feature_vuln_path="../gpt2_defects4j/layer1/feature_importance_vuln.jsonl",
    humaneval_feature_safe_path="../gpt2_humaneval/layer1/feature_importance_safe.jsonl",
    humaneval_feature_vuln_path="../gpt2_humaneval/layer1/feature_importance_vuln.jsonl",
    model_path = "models/gbug_decision_tree_layer10_k_76.pt",
    layer = 10,
    top_k = 76,
    dataset = GBUG_DATASET,
)


In [ ]:

def read_jsonl_file(jsonl_path):
    with open(jsonl_path, "r") as f:
        for line in f:
            yield json.loads(line)

def get_feature_diff_path(dataset: Dataset, training_indexes):
    """
    Get the feature diff path for the given dataset.
    """
    import warnings
    warnings.filterwarnings("ignore")
    diff_data = list(read_jsonl_file(dataset.feature_diff_path))
    diff_df = pd.DataFrame(diff_data)

    columns = diff_df.columns.to_list()
    columns.remove("values")

    diff_df.drop(columns=columns, inplace=True)

    for i in trange(len(diff_df["values"][0])):
        diff_df[f"feature_{i}"] = diff_df["values"].apply(lambda x: x[i])

    diff_df.drop(columns=["values"], inplace=True)

    
    return diff_df



def get_most_important_features(train_df_diff, n=100):
    features = train_df_diff.sum(axis=0).sort_values(ascending=False).index[1 : n + 1]
    return features


In [ ]:
OUR_CONFIG = SAE_ACTIVATIONS_GBUG_LAYER_1

In [ ]:
_, test_df = get_testing_dataset(OUR_CONFIG.dataset)

with open(OUR_CONFIG.dataset.training_idx_path, "r") as f:
    training_indices = json.load(f)

train_df_diff = get_feature_diff_path(
    SAE_ACTIVATIONS_GBUG_LAYER_1, training_indices
)

In [ ]:
top_k = get_most_important_features(train_df_diff, n=OUR_CONFIG.top_k)

In [ ]:
def get_vuln_safe_data(vuln_jsonl_path, safe_jsonl_path, train_indexes):
    vuln_data = list(read_jsonl_file(vuln_jsonl_path))
    safe_data = list(read_jsonl_file(safe_jsonl_path))
    vuln_df = pd.DataFrame(vuln_data)
    safe_df = pd.DataFrame(safe_data)
    vuln_df.drop(columns=["labels", "model", "plot_type"], inplace=True)
    vuln_df["vuln"] = 1

    safe_df.drop(columns=["labels", "model", "plot_type"], inplace=True)
    safe_df["vuln"] = 0

    for i in trange(len(vuln_df["values"][0])):
        vuln_df[f"feature_{i}"] = vuln_df["values"].apply(lambda x: x[i])
        safe_df[f"feature_{i}"] = safe_df["values"].apply(lambda x: x[i])

    safe_df_train = safe_df.loc[train_indexes]
    safe_df_test = safe_df.drop(train_indexes)

    vuln_df_train = vuln_df.loc[train_indexes]
    vuln_df_test = vuln_df.drop(train_indexes)

    df_train = pd.concat([safe_df_train, vuln_df_train])
    df_test = pd.concat([safe_df_test, vuln_df_test])

    df_train = df_train.sample(frac=1).reset_index(drop=True)
    df_test = df_test.sample(frac=1).reset_index(drop=True)
    df_train.drop(columns=["values"], inplace=True)
    df_test.drop(columns=["values"], inplace=True)

    return df_train, df_test


In [ ]:
def evaluate_clf(clf, test_df):
    """
    Evaluate the model on the test dataset.
    """
    #bug_id	func_before	func_after
    tp = 0
    fp = 0
    fn = 0
    tn = 0

    for i, row in tqdm(test_df.iterrows()):
        X = row[top_k.tolist()]
        y = row["vuln"]

        # Get the prediction
        pred = clf.predict([X])

        # Update the confusion matrix
        if pred == 1 and y == 1:
            tp += 1
        elif pred == 1 and y == 0:
            fp += 1
        elif pred == 0 and y == 1:
            fn += 1
        elif pred == 0 and y == 0:
            tn += 1


    # Calculate the accuracy
    accuracy = (tp + tn) / (tp + fp + fn + tn)
    # Calculate the precision
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    # Calculate the recall
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    # Calculate the F1 score
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    
    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1,
        "tp": tp,
        "fp": fp,
        "fn": fn,
        "tn": tn
    }

In [ ]:
with open(SAE_ACTIVATIONS_GBUG_LAYER_1.model_path, "rb") as f:
    clf = pickle.load(f)

# Original (GBUG)
_, df_test = get_vuln_safe_data(
    OUR_CONFIG.gbug_feature_vuln_path,
    OUR_CONFIG.gbug_feature_safe_path,
    training_indices,
)
# filter columns based on top_k
df_test_filtered = df_test[
    top_k.tolist() + ["vuln"]
]

results = evaluate_clf(clf, df_test_filtered)
results

In [ ]:
# Defects4J
_, df_test = get_vuln_safe_data(
    OUR_CONFIG.defects_feature_vuln_path,
    OUR_CONFIG.defects_feature_safe_path,
    training_indices,
)
# filter columns based on top_k
df_test_filtered = df_test[
    top_k.tolist() + ["vuln"]
]

results = evaluate_clf(clf, df_test_filtered)
results

In [ ]:
# Defects4J
_, df_test = get_vuln_safe_data(
    OUR_CONFIG.humaneval_feature_vuln_path,
    OUR_CONFIG.humaneval_feature_safe_path,
    training_indices,
)
# filter columns based on top_k
df_test_filtered = df_test[
    top_k.tolist() + ["vuln"]
]

results = evaluate_clf(clf, df_test_filtered)
results

# Baselines Transferability

In [ ]:
@dataclass
class BaselineClassifier:
    path: str
    tfidf_vectorizer_path: str
    base_dataset: Dataset
    input_size: int

DEFECTS4J_KNN_BASELINE = BaselineClassifier(
    path="../ole/defects4j_knn_k_5000.pt",
    tfidf_vectorizer_path = "../artifacts/vectorizer.pkl",
    base_dataset=DEFECT_DATASET,
    input_size=5000,
)
GBUG_KNN_BASELINE = BaselineClassifier(
    path="../ole/gbug_knn_k_5000.pt",
    tfidf_vectorizer_path = "../artifacts/vectorizer.pkl",
    base_dataset=GBUG_DATASET,
    input_size=5000,
)
HUMANEVAL_KNN_BASELINE = BaselineClassifier(
    path="../ole/human_knn_k_5000.pt",
    tfidf_vectorizer_path = "../artifacts/vectorizer.pkl",
    base_dataset=HUMAN_DATASET,
    input_size=5000,
)
DEFECTS4J_RF_BASELINE = BaselineClassifier(
    path="../ole/defects4j_random_forest_k_5000.pt",
    tfidf_vectorizer_path = "../artifacts/vectorizer.pkl",
    base_dataset=DEFECT_DATASET,
    input_size=5000,
)
GBUG_RF_BASELINE = BaselineClassifier(
    path="../ole/gbug_random_forest_k_5000.pt",
    tfidf_vectorizer_path = "../artifacts/vectorizer.pkl",
    base_dataset=GBUG_DATASET,
    input_size=5000,
)
HUMANEVAL_RF_BASELINE = BaselineClassifier(
    path="../ole/human_random_forest_k_5000.pt",
    tfidf_vectorizer_path = "../artifacts/vectorizer.pkl",
    base_dataset=HUMAN_DATASET,
    input_size=5000,
)

def load_baseline_classifier(classifier: BaselineClassifier):
    with open(classifier.path, "rb") as f:
        clf = pickle.load(f)
    return clf
@dataclass
class Results:
    precision: float
    recall: float
    accuracy: float
    f1: float


@dataclass
class TransferabilityPerformance:
    on_defects: Optional[int] = None
    on_humaneval: Optional[int] = None
    on_gbug: Optional[int] = None

def calculate_f1_shift(
    results: Results,
    baseline_results: Results,
) -> dict:
    """
    Calculate the shift in perfoamcene in terms of F1 score % changed
    """
    f1_shift = (results.f1 - baseline_results.f1) / baseline_results.f1 * 100
    
    return f1_shift

def test_baseline_classifier(clf, 
                             vectorizer, 
                             df, 
                             train_indexes,
                             before_func_col="func_before", 
                             after_func_col="func_after") -> Results:
    """
    Test the baseline classifier on the test dataset.
    """

    df["tokenized_before"] = df[before_func_col].progress_apply(
        lambda x: vectorizer.transform([x]).toarray()[0]
    )
    df["tokenized_after"] = df[after_func_col].progress_apply(
        lambda x: vectorizer.transform([x]).toarray()[0]
    )
    # Pad to 5000 tokens
    df["tokenized_before"] = df["tokenized_before"].apply(
        lambda x: x[:5000] + [0] * (5000 - len(x)) if len(x) < 5000 else x[:5000]
    )
    df["tokenized_after"] = df["tokenized_after"].apply(
        lambda x: x[:5000] + [0] * (5000 - len(x)) if len(x) < 5000 else x[:5000]
    )

    df_test = df.drop(train_indexes)

    df_classical_test = pd.DataFrame()
    for row in df_test.iterrows():
        row = row[1]
        df_classical_test = pd.concat(
            [
                df_classical_test,
                pd.DataFrame(
                    {"tokens": [row["tokenized_before"].tolist()], "vuln": 1}, index=[0]
                ),
            ]
        )
        df_classical_test = pd.concat(
            [
                df_classical_test,
                pd.DataFrame(
                    {"tokens": [row["tokenized_after"].tolist()], "vuln": 0}, index=[0]
                ),
            ]
        )

    X_test = df_classical_test["tokens"].values.tolist()
    X_test = [torch.tensor(x) for x in X_test]
    y_test = df_classical_test["vuln"]

    # Get the prediction
    y_pred = clf.predict(X_test)
    precision, recall, accuracy, f1 = get_metrics(y_pred, y_test)
    logger.info(
        "Classification report:",
        precision=precision,
        recall=recall,
        accuracy=accuracy,
        f1=f1,
    )
    return Results(
        precision=precision,
        recall=recall,
        accuracy=accuracy,
        f1=f1,
    )
    



In [78]:

results_gbug_knn = test_baseline_classifier(
    load_baseline_classifier(GBUG_KNN_BASELINE),
    load_tfidf_vectorizer(GBUG_KNN_BASELINE.tfidf_vectorizer_path),
    pd.read_csv(GBUG_DATASET.dataset_path),
    load_training_indexes(GBUG_DATASET.training_idx_path),
)

results_gbug_knn_defects = test_baseline_classifier(
    load_baseline_classifier(GBUG_KNN_BASELINE),
    load_tfidf_vectorizer(GBUG_KNN_BASELINE.tfidf_vectorizer_path),
    pd.read_csv(DEFECT_DATASET.dataset_path),
    load_training_indexes(DEFECT_DATASET.training_idx_path),
)

results_gbug_knn_human = test_baseline_classifier(
    load_baseline_classifier(GBUG_KNN_BASELINE),
    load_tfidf_vectorizer(GBUG_KNN_BASELINE.tfidf_vectorizer_path),
    pd.read_csv(HUMAN_DATASET.dataset_path),
    load_training_indexes(HUMAN_DATASET.training_idx_path),
)


transf_knn_gbug_on_defects= calculate_f1_shift(
    results_gbug_knn_defects,
    results_gbug_knn,
)
transf_knn_gbug_on_human= calculate_f1_shift(
    results_gbug_knn_human,
    results_gbug_knn,
)

transf_knn_gbug = TransferabilityPerformance(
    on_defects=transf_knn_gbug_on_defects,
    on_humaneval=transf_knn_gbug_on_human,
)

  0%|          | 0/148 [00:00<?, ?it/s]

100%|██████████| 148/148 [00:00<00:00, 5137.10it/s]

2025-04-21 14:51:13 [info     ] Classification report:         accuracy=0.6666666666666666 f1=0.6774193548387096 precision=0.65625 recall=0.7



100%|██████████| 465/465 [00:00<00:00, 6656.83it/s]


2025-04-21 14:51:13 [info     ] Classification report:         accuracy=0.5 f1=0.4431137724550898 precision=0.5 recall=0.3978494623655914


100%|██████████| 162/162 [00:00<00:00, 7150.44it/s]


2025-04-21 14:51:13 [info     ] Classification report:         accuracy=0.48484848484848486 f1=0.43333333333333324 precision=0.48148148148148145 recall=0.3939393939393939


In [79]:
### Random Forest from Gbug
results_gbug_rf = test_baseline_classifier(
    load_baseline_classifier(GBUG_RF_BASELINE),
    load_tfidf_vectorizer(GBUG_RF_BASELINE.tfidf_vectorizer_path),
    pd.read_csv(GBUG_DATASET.dataset_path),
    load_training_indexes(GBUG_DATASET.training_idx_path),
)

results_gbug_rf_defects = test_baseline_classifier(
    load_baseline_classifier(GBUG_RF_BASELINE),
    load_tfidf_vectorizer(GBUG_RF_BASELINE.tfidf_vectorizer_path),
    pd.read_csv(DEFECT_DATASET.dataset_path),
    load_training_indexes(DEFECT_DATASET.training_idx_path),
)

results_gbug_rf_human = test_baseline_classifier(
    load_baseline_classifier(GBUG_KNN_BASELINE),
    load_tfidf_vectorizer(GBUG_KNN_BASELINE.tfidf_vectorizer_path),
    pd.read_csv(HUMAN_DATASET.dataset_path),
    load_training_indexes(HUMAN_DATASET.training_idx_path),
)

transf_rf_gbug_on_defects= calculate_f1_shift(
    results_gbug_rf_defects,
    results_gbug_rf,
)
transf_rf_gbug_on_human= calculate_f1_shift(
    results_gbug_rf_human,
    results_gbug_rf,
)

transf_rf_gbug = TransferabilityPerformance(
    on_defects=transf_rf_gbug_on_defects,
    on_humaneval=transf_rf_gbug_on_human,
)

  0%|          | 0/148 [00:00<?, ?it/s]

100%|██████████| 148/148 [00:00<00:00, 4205.02it/s]

2025-04-21 14:51:18 [info     ] Classification report:         accuracy=0.75 f1=0.7058823529411764 precision=0.8571428571428571 recall=0.6



100%|██████████| 465/465 [00:00<00:00, 6809.41it/s]


2025-04-21 14:51:18 [info     ] Classification report:         accuracy=0.5161290322580645 f1=0.1176470588235294 precision=0.6666666666666666 recall=0.06451612903225806


100%|██████████| 162/162 [00:00<00:00, 8123.93it/s]


2025-04-21 14:51:18 [info     ] Classification report:         accuracy=0.48484848484848486 f1=0.43333333333333324 precision=0.48148148148148145 recall=0.3939393939393939


In [80]:
results_defects_knn = test_baseline_classifier(
    load_baseline_classifier(DEFECTS4J_KNN_BASELINE),
    load_tfidf_vectorizer(DEFECTS4J_KNN_BASELINE.tfidf_vectorizer_path),
    pd.read_csv(DEFECT_DATASET.dataset_path),
    load_training_indexes(DEFECT_DATASET.training_idx_path),
)
results_defects_knn_gbug = test_baseline_classifier(
    load_baseline_classifier(DEFECTS4J_KNN_BASELINE),
    load_tfidf_vectorizer(DEFECTS4J_KNN_BASELINE.tfidf_vectorizer_path),
    pd.read_csv(GBUG_DATASET.dataset_path),
    load_training_indexes(GBUG_DATASET.training_idx_path),
)
results_defects_knn_human = test_baseline_classifier(
    load_baseline_classifier(DEFECTS4J_KNN_BASELINE),
    load_tfidf_vectorizer(DEFECTS4J_KNN_BASELINE.tfidf_vectorizer_path),
    pd.read_csv(HUMAN_DATASET.dataset_path),
    load_training_indexes(HUMAN_DATASET.training_idx_path),
)
transf_knn_defects_on_gbug= calculate_f1_shift(
    results_defects_knn_gbug,
    results_defects_knn,
)
transf_knn_defects_on_human= calculate_f1_shift(
    results_defects_knn_human,
    results_defects_knn,
)

transf_knn_defects = TransferabilityPerformance(
    on_defects=transf_knn_defects_on_gbug,
    on_humaneval=transf_knn_defects_on_human,
)


  0%|          | 0/465 [00:00<?, ?it/s]

100%|██████████| 465/465 [00:00<00:00, 6766.91it/s]


2025-04-21 14:51:23 [info     ] Classification report:         accuracy=0.5376344086021505 f1=0.35820895522388063 precision=0.5853658536585366 recall=0.25806451612903225


100%|██████████| 148/148 [00:00<00:00, 5345.32it/s]


2025-04-21 14:51:23 [info     ] Classification report:         accuracy=0.5333333333333333 f1=0.43999999999999995 precision=0.55 recall=0.36666666666666664


100%|██████████| 162/162 [00:00<00:00, 7565.80it/s]


2025-04-21 14:51:23 [info     ] Classification report:         accuracy=0.5454545454545454 f1=0.4642857142857143 precision=0.5652173913043478 recall=0.3939393939393939


In [81]:
results_defects_rf = test_baseline_classifier(
    load_baseline_classifier(DEFECTS4J_RF_BASELINE),
    load_tfidf_vectorizer(DEFECTS4J_RF_BASELINE.tfidf_vectorizer_path),
    pd.read_csv(DEFECT_DATASET.dataset_path),
    load_training_indexes(DEFECT_DATASET.training_idx_path),
)
results_defects_rf_gbug = test_baseline_classifier(
    load_baseline_classifier(DEFECTS4J_RF_BASELINE),
    load_tfidf_vectorizer(DEFECTS4J_RF_BASELINE.tfidf_vectorizer_path),
    pd.read_csv(GBUG_DATASET.dataset_path),
    load_training_indexes(GBUG_DATASET.training_idx_path),
)
results_defects_rf_human = test_baseline_classifier(
    load_baseline_classifier(DEFECTS4J_RF_BASELINE),
    load_tfidf_vectorizer(DEFECTS4J_RF_BASELINE.tfidf_vectorizer_path),
    pd.read_csv(HUMAN_DATASET.dataset_path),
    load_training_indexes(HUMAN_DATASET.training_idx_path),
)
transf_rf_defects_on_gbug= calculate_f1_shift(
    results_defects_rf_gbug,
    results_defects_rf,
)
transf_rf_defects_on_human= calculate_f1_shift(
    results_defects_rf_human,
    results_defects_rf,
)

transf_rf_defects = TransferabilityPerformance(
    on_defects=transf_rf_defects_on_gbug,
    on_humaneval=transf_rf_defects_on_human,
)

  0%|          | 0/465 [00:00<?, ?it/s]

100%|██████████| 465/465 [00:00<00:00, 7023.51it/s]


2025-04-21 14:51:28 [info     ] Classification report:         accuracy=0.521505376344086 f1=0.41830065359477125 precision=0.5333333333333333 recall=0.34408602150537637


100%|██████████| 148/148 [00:00<00:00, 5355.74it/s]


2025-04-21 14:51:28 [info     ] Classification report:         accuracy=0.5333333333333333 f1=0.36363636363636365 precision=0.5714285714285714 recall=0.26666666666666666


100%|██████████| 162/162 [00:00<00:00, 8207.54it/s]


2025-04-21 14:51:28 [info     ] Classification report:         accuracy=0.5 f1=0.6117647058823529 precision=0.5 recall=0.7878787878787878


In [82]:
results_humaneval_knn = test_baseline_classifier(
    load_baseline_classifier(HUMANEVAL_KNN_BASELINE),
    load_tfidf_vectorizer(HUMANEVAL_KNN_BASELINE.tfidf_vectorizer_path),
    pd.read_csv(HUMAN_DATASET.dataset_path),
    load_training_indexes(HUMAN_DATASET.training_idx_path),
)
results_humaneval_knn_gbug = test_baseline_classifier(
    load_baseline_classifier(HUMANEVAL_KNN_BASELINE),
    load_tfidf_vectorizer(HUMANEVAL_KNN_BASELINE.tfidf_vectorizer_path),
    pd.read_csv(GBUG_DATASET.dataset_path),
    load_training_indexes(GBUG_DATASET.training_idx_path),
)
results_humaneval_knn_defects = test_baseline_classifier(
    load_baseline_classifier(HUMANEVAL_KNN_BASELINE),
    load_tfidf_vectorizer(HUMANEVAL_KNN_BASELINE.tfidf_vectorizer_path),
    pd.read_csv(DEFECT_DATASET.dataset_path),
    load_training_indexes(DEFECT_DATASET.training_idx_path),
)
transf_knn_humaneval_on_gbug= calculate_f1_shift(
    results_humaneval_knn_gbug,
    results_humaneval_knn,
)
transf_knn_humaneval_on_defects= calculate_f1_shift(
    results_humaneval_knn_defects,
    results_humaneval_knn,
)

transf_knn_humaneval = TransferabilityPerformance(
    on_defects=transf_knn_humaneval_on_defects,
    on_gbug=transf_knn_humaneval_on_gbug,
)


  0%|          | 0/162 [00:00<?, ?it/s]

100%|██████████| 162/162 [00:00<00:00, 5098.69it/s]

2025-04-21 14:51:32 [info     ] Classification report:         accuracy=0.5757575757575758 f1=0.5625 precision=0.5806451612903226 recall=0.5454545454545454



100%|██████████| 148/148 [00:00<00:00, 5104.87it/s]


2025-04-21 14:51:32 [info     ] Classification report:         accuracy=0.48333333333333334 f1=0.5373134328358209 precision=0.4864864864864865 recall=0.6


100%|██████████| 465/465 [00:00<00:00, 6909.96it/s]


2025-04-21 14:51:33 [info     ] Classification report:         accuracy=0.5053763440860215 f1=0.5 precision=0.5054945054945055 recall=0.4946236559139785


In [83]:
results_defects_rf = test_baseline_classifier(
    load_baseline_classifier(DEFECTS4J_RF_BASELINE),
    load_tfidf_vectorizer(DEFECTS4J_RF_BASELINE.tfidf_vectorizer_path),
    pd.read_csv(DEFECT_DATASET.dataset_path),
    load_training_indexes(DEFECT_DATASET.training_idx_path),
)
results_defects_rf_gbug = test_baseline_classifier(
    load_baseline_classifier(DEFECTS4J_RF_BASELINE),
    load_tfidf_vectorizer(DEFECTS4J_RF_BASELINE.tfidf_vectorizer_path),
    pd.read_csv(GBUG_DATASET.dataset_path),
    load_training_indexes(GBUG_DATASET.training_idx_path),
)
results_defects_rf_human = test_baseline_classifier(
    load_baseline_classifier(DEFECTS4J_RF_BASELINE),
    load_tfidf_vectorizer(DEFECTS4J_RF_BASELINE.tfidf_vectorizer_path),
    pd.read_csv(HUMAN_DATASET.dataset_path),
    load_training_indexes(HUMAN_DATASET.training_idx_path),
)
transf_rf_defects_on_gbug= calculate_f1_shift(
    results_defects_rf_gbug,
    results_defects_rf,
)
transf_rf_defects_on_human= calculate_f1_shift(
    results_defects_rf_human,
    results_defects_rf,
)

transf_rf_defects = TransferabilityPerformance(
    on_defects=transf_rf_defects_on_gbug,
    on_humaneval=transf_rf_defects_on_human,
)


  0%|          | 0/465 [00:00<?, ?it/s]

100%|██████████| 465/465 [00:00<00:00, 7038.62it/s]


2025-04-21 14:51:37 [info     ] Classification report:         accuracy=0.521505376344086 f1=0.41830065359477125 precision=0.5333333333333333 recall=0.34408602150537637


100%|██████████| 148/148 [00:00<00:00, 5248.38it/s]


2025-04-21 14:51:38 [info     ] Classification report:         accuracy=0.5333333333333333 f1=0.36363636363636365 precision=0.5714285714285714 recall=0.26666666666666666


100%|██████████| 162/162 [00:00<00:00, 7663.94it/s]


2025-04-21 14:51:38 [info     ] Classification report:         accuracy=0.5 f1=0.6117647058823529 precision=0.5 recall=0.7878787878787878


In [84]:
results_humaneval_knn = test_baseline_classifier(
    load_baseline_classifier(HUMANEVAL_KNN_BASELINE),
    load_tfidf_vectorizer(HUMANEVAL_KNN_BASELINE.tfidf_vectorizer_path),
    pd.read_csv(HUMAN_DATASET.dataset_path),
    load_training_indexes(HUMAN_DATASET.training_idx_path),
)
results_humaneval_knn_gbug = test_baseline_classifier(
    load_baseline_classifier(HUMANEVAL_KNN_BASELINE),
    load_tfidf_vectorizer(HUMANEVAL_KNN_BASELINE.tfidf_vectorizer_path),
    pd.read_csv(GBUG_DATASET.dataset_path),
    load_training_indexes(GBUG_DATASET.training_idx_path),
)
results_humaneval_knn_defects = test_baseline_classifier(
    load_baseline_classifier(HUMANEVAL_KNN_BASELINE),
    load_tfidf_vectorizer(HUMANEVAL_KNN_BASELINE.tfidf_vectorizer_path),
    pd.read_csv(DEFECT_DATASET.dataset_path),
    load_training_indexes(DEFECT_DATASET.training_idx_path),
)
transf_knn_humaneval_on_gbug= calculate_f1_shift(
    results_humaneval_knn_gbug,
    results_humaneval_knn,
)
transf_knn_humaneval_on_defects= calculate_f1_shift(
    results_humaneval_knn_defects,
    results_humaneval_knn,
)
transf_knn_humaneval = TransferabilityPerformance(
    on_defects=transf_knn_humaneval_on_defects,
    on_gbug=transf_knn_humaneval_on_gbug,
)


100%|██████████| 162/162 [00:00<00:00, 7700.33it/s]

2025-04-21 14:54:18 [info     ] Classification report:         accuracy=0.5757575757575758 f1=0.5625 precision=0.5806451612903226 recall=0.5454545454545454



100%|██████████| 148/148 [00:00<00:00, 5489.83it/s]


2025-04-21 14:54:19 [info     ] Classification report:         accuracy=0.48333333333333334 f1=0.5373134328358209 precision=0.4864864864864865 recall=0.6


100%|██████████| 465/465 [00:00<00:00, 6857.87it/s]


2025-04-21 14:54:19 [info     ] Classification report:         accuracy=0.5053763440860215 f1=0.5 precision=0.5054945054945055 recall=0.4946236559139785


In [85]:
results_humaneval_rf = test_baseline_classifier(
    load_baseline_classifier(HUMANEVAL_RF_BASELINE),
    load_tfidf_vectorizer(HUMANEVAL_RF_BASELINE.tfidf_vectorizer_path),
    pd.read_csv(HUMAN_DATASET.dataset_path),
    load_training_indexes(HUMAN_DATASET.training_idx_path),
)
results_humaneval_rf_gbug = test_baseline_classifier(
    load_baseline_classifier(HUMANEVAL_RF_BASELINE),
    load_tfidf_vectorizer(HUMANEVAL_RF_BASELINE.tfidf_vectorizer_path),
    pd.read_csv(GBUG_DATASET.dataset_path),
    load_training_indexes(GBUG_DATASET.training_idx_path),
)
results_humaneval_rf_defects = test_baseline_classifier(
    load_baseline_classifier(HUMANEVAL_RF_BASELINE),
    load_tfidf_vectorizer(HUMANEVAL_RF_BASELINE.tfidf_vectorizer_path),
    pd.read_csv(DEFECT_DATASET.dataset_path),
    load_training_indexes(DEFECT_DATASET.training_idx_path),
)
transf_rf_humaneval_on_gbug= calculate_f1_shift(
    results_humaneval_rf_gbug,
    results_humaneval_rf,
)
transf_rf_humaneval_on_defects= calculate_f1_shift(
    results_humaneval_rf_defects,
    results_humaneval_rf,
)
transf_rf_humaneval = TransferabilityPerformance(
    on_defects=transf_rf_humaneval_on_defects,
    on_gbug=transf_rf_humaneval_on_gbug,
)

100%|██████████| 162/162 [00:00<00:00, 7315.41it/s]

2025-04-21 14:54:51 [info     ] Classification report:         accuracy=0.5606060606060606 f1=0.4912280701754386 precision=0.5833333333333334 recall=0.42424242424242425



100%|██████████| 148/148 [00:00<00:00, 5492.11it/s]

2025-04-21 14:54:51 [info     ] Classification report:         accuracy=0.5 f1=0.16666666666666669 precision=0.5 recall=0.1



100%|██████████| 465/465 [00:00<00:00, 6482.01it/s]


2025-04-21 14:54:51 [info     ] Classification report:         accuracy=0.4946236559139785 f1=0.11320754716981131 precision=0.46153846153846156 recall=0.06451612903225806


In [99]:
# plot the different shift values
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
sns.set(style="whitegrid")
# We have 3 datasets: Gbug, Defects4J, and HumanEval
# We have 2 classifiers: KNN, RF
# Plot the different shift values for each dataset and classifier
fig, ax = plt.subplots(2, 3, figsize=(15, 5))
fig.suptitle("Transferability Performance of KNN and RF Classifiers", fontsize=16)
# Gbug
ax[0, 0].bar(
    ["Defects4J", "HumanEval"],
    [transf_knn_gbug.on_defects, transf_knn_gbug.on_humaneval],
)

ax[0, 0].set_title("Gbug KNN")
ax[0, 0].set_ylabel("F1 Shift (%)")
ax[0, 0].set_ylim(-100, 100)
ax[0, 0].set_xticklabels(["Defects4J", "HumanEval"], rotation=45)
# Defects4J
ax[0, 1].bar(
    ["Gbug", "HumanEval"],
    [transf_knn_defects.on_defects, transf_knn_defects.on_humaneval],
)
ax[0, 1].set_title("Defects4J KNN")
ax[0, 1].set_ylabel("F1 Shift (%)")
ax[0, 1].set_ylim(-100, 100)
ax[0, 1].set_xticklabels(["Gbug", "HumanEval"], rotation=45)
# HumanEval
ax[0, 2].bar(
    ["Gbug", "Defects4J"],
    [transf_knn_humaneval.on_gbug, transf_knn_humaneval.on_defects],
)
ax[0, 2].set_title("HumanEval KNN")
ax[0, 2].set_ylabel("F1 Shift (%)")
ax[0, 2].set_ylim(-100, 100)
ax[0, 2].set_xticklabels(["Gbug", "Defects4J"], rotation=45)
# Gbug
ax[1, 0].bar(
    ["Defects4J", "HumanEval"],
    [transf_rf_gbug.on_defects, transf_rf_gbug.on_humaneval],
)
ax[1, 0].set_title("Gbug RF")
ax[1, 0].set_ylabel("F1 Shift (%)")
ax[1, 0].set_ylim(-100, 100)
ax[1, 0].set_xticklabels(["Defects4J", "HumanEval"], rotation=45)
# Defects4J
ax[1, 1].bar(
    ["Gbug", "HumanEval"],
    [transf_rf_defects.on_defects, transf_rf_defects.on_humaneval],
)
ax[1, 1].set_title("Defects4J RF")
ax[1, 1].set_ylabel("F1 Shift (%)")
ax[1, 1].set_ylim(-100, 100)
ax[1, 1].set_xticklabels(["Gbug", "HumanEval"], rotation=45)
# HumanEval
ax[1, 2].bar(
    ["Gbug", "Defects4J"],
    [transf_rf_humaneval.on_gbug, transf_rf_humaneval.on_defects],
)
ax[1, 2].set_title("HumanEval RF")
ax[1, 2].set_ylabel("F1 Shift (%)")
ax[1, 2].set_ylim(-100, 100)
ax[1, 2].set_xticklabels(["Gbug", "Defects4J"], rotation=45)
plt.tight_layout()
plt.subplots_adjust(top=0.85)
plt.show()

# Save the figure
fig.savefig("transferability_performance.png", dpi=300)
